# Uczenie gradientowe

### Rozbicie warstw na warstwę liniową oraz warstwę aktywacji

Tradycyjnie przez neuron rozumie się funkcję
$$o(x) = a(<x,w> + b)$$
gdzie $w$ to wagi, $b$ to bias, natomiast $a$ to funkcja aktywacji. Grupa neuronów mających wspólne wejście $x$ tworzy warstwę sieci neuronowej.

<img src="figures/L10/nn.png" width=600>

Znacznie wygodniej jest rozumieć warstwy tak, jak w bibliotece *keras*, tzn. rozważając oddzielnie część liniową i funkcję aktywacji. Wtedy neuron w tradycyjnym rozumieniu jest tak naprawdę parą kolejnych neuronów
$$o(x) = o_2(o_1(x))$$
$$o_1(x) = <x|w> + b$$
$$o_2(x) = a(x)$$

<img src="figures/L10/nn_extended.png" width=600>

Neurony liniowe tworzą warstwę *Dense* (połączenia typu "każdy z każdym" pomiędzy wejściem i wyjściem) sparametryzowaną macierzą wag $W$, natomiast neurony aktywacji tworzą warstwę aktywacji (połączenia typu "jeden do jeden") i warstwa ta nie posiada żadnych wag.


### Uczenie metodami gradientowymi

Najprostszy algorytm uczenia sieci neuronowej:
1. Inicjalizujemy wagi (np. losowo).
2. W pętli:
    1. Obliczamy wartość funkcji kosztu.
    2. Dokonujemy losowej modyfikacji wag i ponownie obliczamy wartość funkcji kosztu:
        1. Jeśli uległa ona zmniejszeniu, zmieniamy wagi na nowe.
        2. Jeśli uległa zwiększeniu, zostajemy przy starych wagach.

Powyższy algorytm ma poważną wadę: losowa modyfikacja bardzo dużej liczby parametrów (w wypadku sieci neuronowych są to liczby rzędu kilku milionów) spowoduje niewielką zmianę funkcji kosztu - intuicyjnie możemy myśleć, że zmiany spowodowane przez modyfikację kolejnych parametrów będą się wzajemnie kasować.

Metody gradientowe działają następująco:
1. Inicjalizujemy wagi (np. losowo).
2. W pętli:
    1. Dla każdego parametru sieci obliczamy, jak silny jest jego wpływ na wartość funkcji kosztu.
    2. Na tej podstawie modyfikujemy parametry.

Szczegóły kroku 2B zależą od konkretnej metody. 

Przez siłę wpływu parametru sieci neuronowej na wartość funkcji kosztu rozumiemy pochodną cząstkową $\frac{\partial L}{\partial\theta}$, gdzie $\theta$ jest dowolnym parametrem (wagą), a $L$ funkcją kosztu, która zależy od parametrów sieci i elementów zbioru treningowego. Taka definicja ma sens, ponieważ (powtórka z analizy matematycznej):
$$L(\ldots, \theta + \Delta\theta, \ldots) \approx L(\ldots, \theta, \ldots) + \frac{\partial L}{\partial\theta}\Delta\theta$$
a więc zmiana parametru $\theta$ o $\Delta\theta$ spowoduje $\frac{\partial L}{\partial\theta}$ razy większą zmianę kosztu $L$.

Najefektywniej modyfikować parametry **proporcjonalnie do siły ich wpływu**, ponieważ wtedy poruszamy się w kierunku najszybszej zmiany wartości funkcji $L$. Kierunek ten wyznaczony jest (minus) gradientem funkcji $L$.

### Wpływ pojedynczego neuronu

<img src="figures/L10/single_neuron.png" width=430>

Aby obliczyć wpływ neuronu na wartość funkcji kosztu musimy:
1. obliczyć wpływ tego neuronu na kolejne neurony,
2. obliczyć wpływ kolejnych neuronów na wartość funkcji kosztu.

$$\frac{\partial L}{\partial o} = \frac{\partial L}{\partial o_1}\frac{\partial o_1}{\partial o} + \frac{\partial L}{\partial o_2}\frac{\partial o_2}{\partial o} + \frac{\partial L}{\partial o_3}\frac{\partial o_3}{\partial o}$$

Jak rozumieć powyższy wzór:
* jeśli np. $\frac{\partial L}{\partial o_1} = 2$, oznacza to, że mała zmiana $o_1$ spowoduje dwukrotnie większą zmianę $L$,
* jeśli np. $\frac{\partial o_1}{\partial o} = 3$, oznacza to, że mała zmiana $o$ spowoduje trzykrotnie większą zmianę $o_1$,
* łącząc dwa powyższe fakty wnioskujemy, że mała zmiana $o$ spowoduje sześciokrotnie (trzy razy dwa) większą zmianę $L$, stąd mnożenie $\frac{\partial L}{\partial o_1}\frac{\partial o_1}{\partial o}$,
* skoro $o$ wpływa bezpośrednio na $o_1$, $o_2$ i $o_3$, to zmiany wartości $L$ spowodowane przez $o$ przy pomocy  $o_1$, $o_2$ i $o_3$ należy zsumować,
* $o$ nie wpływa na $L$ w żaden inny sposób, dlatego po prawej stronie równości $\frac{\partial L}{\partial o} = \frac{\partial L}{\partial o_1}\frac{\partial o_1}{\partial o} + \frac{\partial L}{\partial o_2}\frac{\partial o_2}{\partial o} + \frac{\partial L}{\partial o_3}\frac{\partial o_3}{\partial o}$ nie ma nic więcej.



<img src="figures/L10/single_neuron_paths.png" width=430>

Jeszcze inaczej - powyższy wzór opisuje wszystkie (skierowane) ścieżki, którymi możemy dostać się z $o$ do $L$:
* $\frac{\partial L}{\partial o}$ - wszystkie ścieżki z $o$ do $L$,
* $\frac{\partial L}{\partial o_1}$ - wszystkie ścieżki z $o_1$ do $L$,
* $\frac{\partial L}{\partial o_2}$ - wszystkie ścieżki z $o_2$ do $L$,
* $\frac{\partial L}{\partial o_3}$ - wszystkie ścieżki z $o_3$ do $L$,
* $\frac{\partial o_1}{\partial o}$ - wszystkie ścieżki (w tym wypadku dokładnie jedna) z $o$ do $o_1$,
* $\frac{\partial o_2}{\partial o}$ - wszystkie ścieżki (w tym wypadku dokładnie jedna) z $o$ do $o_2$,
* $\frac{\partial o_3}{\partial o}$ - wszystkie ścieżki (w tym wypadku dokładnie jedna) z $o$ do $o_3$.

Mnożenie to konkatenacja ścieżek (każda z każdą), dodawanie to suma zbiorów ścieżek. Jeśli po dwóch stronach równości są te same zbiory ścieżek, to równość jest prawdziwa.


### Wpływ pojedynczej wagi

<img src="figures/L10/single_neuron_weight.png" width=430>

Aby obliczyć wpływ wagi na wartość funkcji kosztu musimy:
1. obliczyć wpływ tej wagi na kolejny neuron (zawsze jest dokładnie jeden),
2. obliczyć wpływ kolejnego neuronu na wartość funkcji kosztu.

$\frac{\partial L}{\partial w_1} = \frac{\partial L}{\partial o_1}\frac{\partial o_1}{\partial w_1}$

Powyższy wzór również możemy rozumieć jako zbiór wszystkich ścieżek z krawędzi $w_1$ do $L$ zapisany na dwa różne sposoby.

Przechodząc wstecz przez warstwę aktywacji pomijamy krok opisany w tej sekcji, ponieważ warstwy aktywacji nie mają parametrów (wag).

### Wpływ neuronów z najwyższej warstwy

Neurony z najwyższej warstwy są bezpośrednio połączone z węzłem obliczającym wartość funkcji kosztu $L$, więc pochodne cząstkowe $\frac{\partial L}{\partial o}$ liczymy wprost ze wzoru na $L$. Jeśli definiujemy
$$L(y_{true}, y_{pred}) = \ldots$$
to pamiętamy, że $o = y_{pred}$.

### Metryka vs funkcja kosztu

Metryka - to, co faktycznie chcemy optymalizować.

Funkcja kosztu - różniczkowalna funkcja, którą chcemy minimalizować.

Jeśli metryka jest różniczkowalna, to używamy jej jako funkcji kosztu (być może po drobnych modyfikacjach, aby optymalizacja polegała na minimalizacji). Jeśli nie, to dobieramy funkcję kosztu tak, aby jej minimalizacja przybliżała optymalizację metryki.

Przykład - problem klasyfikacji binarnej:

Nie możemy użyć accuracy jako funkcji kosztu, bo jest ona nieróżniczkowalna. Minimalizacja crossentropy dość sensownie przybliża maksymalizowanie accuracy, więc jest uzasadnione użycie jej jako funkcji kosztu. Oczywiście mogą się zdarzyć przypadki, że zmniejszenie crossentropy pogorszy accuracy, ale z reguły tak nie będzie.

## Backpropagation

Backpropagation to algorytm efektywnego liczenia siły wpływu poszczególnych neuronów oraz wag sieci neuronowej na wartość funkcji kosztu. Polega on na obliczaniu $\frac{\partial L}{\partial o}$ oraz $\frac{\partial L}{\partial w}$ warstwa po warstwie, zaczynając od najwyższej.

Pytanie kontrolne - dlaczego nie zaczynamy od najniższej warstwy? 

We wzorach na pochodne cząstkowe mogą pojawić się wartości innych wag (literka $w$ z indeksami) oraz wartości zwracane przez inne neurony (literka $o$ z indeksami). Wagi już mamy, natomiast wartości neuronów trzeba przeliczyć dla każdego przykładu treningowego $(\mathbf{x},y)$ - w tym celu wykonujemy tzw. *forward pass*, czyli po prostu wstawiamy $\mathbf{x}$ na wejściu sieci i liczymy warstwa po warstwie od dołu.

Pytanie kontrolne - dlaczego nie zaczynamy od najwyższej warstwy?